# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

In [3]:
r = requests.get(url)
r.status_code

200

In [9]:
# 2) Seu código aqui
data = r.json()
df = pd.DataFrame(data['items'])
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,9990,Município,Barra do Piraí,3300308,RJ,None,Arquivado,17944.001095/2008-97,2008-08-18T03:00:00Z,Operação contratual interna,PAC - Saneamento - 2008 B,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1472500.00,0,0,30/10/2008
1,13574,Município,Rio de Janeiro,3304557,RJ,None,Deferido,19405.000028/2006-16,2007-04-30T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,39528049.81,1,0,25/05/2007
2,9858,Município,Teresópolis,3305802,RJ,None,Arquivado,17944.001067/2008-70,2008-09-26T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,709500.00,0,0,10/12/2008
3,8999,Município,Itatiaia,3302254,RJ,None,Arquivado por decurso de prazo,17944.000881/2014-15,2014-07-02T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15000000.00,0,0,18/02/2015
4,13241,Município,Barra Mansa,3300407,RJ,None,Deferido,17944.001974/2006-57,2007-06-11T03:00:00Z,Operação contratual interna,Provias,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1980000.00,1,0,15/06/2007


In [72]:
print('Os 3 status de maior solicitação são:')

for i in range(0,3):
    print('Status: ' + df['status'].value_counts().index.to_list()[i])
    print('Valor:' + df['status'].value_counts()[i].astype(str))

Os 3 status de maior solicitação são:
Status: Deferido
Valor:96
Status: Arquivado
Valor:53
Status: Regularizado
Valor:26


In [ ]:
print('')

In [31]:
# 3) Seu código aqui
df['data_status'] = df['data_status'].astype(str)
df.dtypes

id_pleito                  int64
tipo_interessado          object
interessado               object
cod_ibge                   int64
uf                        object
num_pvl                   object
status                    object
num_processo              object
data_protocolo            object
tipo_operacao             object
finalidade                object
tipo_credor               object
credor                    object
moeda                     object
valor                    float64
pvl_assoc_divida           int64
pvl_contradado_credor      int64
data_status               object
dtype: object

In [37]:
ano = df['data_status'].str.slice(6)
ano

0      2008
1      2007
2      2008
3      2015
4      2007
       ... 
243    2020
244    2007
245    2007
246    2008
247    2013
Name: data_status, Length: 248, dtype: object

In [38]:
ano.value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2023    17
2020    15
2013    15
2012    13
2022    11
2011    10
2009     9
2015     8
2017     5
2021     5
2018     4
2016     4
2002     4
2006     4
2003     1
2004     1
Name: data_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [76]:
#1) Seu código aqui
def busca_uf_interessado(uf, interessado) -> str:
    if interessado == 'M':
        ti = 'Munic%C3%ADpio'
    elif interessado == 'E':
        ti = 'Estado'
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf='+uf+'&tipo_interessado='+ ti
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data['items'])
    return df

In [77]:
busca_uf_interessado('MG', 'E')

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13587,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000031/2004-78,2004-11-27T02:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento rural,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.500000e+07,1,0,09/12/2004
1,13672,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000048/2005-14,2005-12-28T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.700000e+08,1,0,19/04/2006
2,13695,Estado,Minas Gerais,31,MG,None,Arquivado,19405.000053/2003-57,2003-08-25T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,3.500000e+07,0,0,20/02/2004
3,20458,Estado,Minas Gerais,31,MG,00000.000000/2020-68,Arquivado a pedido,17944.001085/2015-81,2015-11-23T02:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,0,0,29/04/2016
4,9197,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000924/2009-03,2010-03-10T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,1,0,01/04/2010
5,13447,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,19405.000002/2006-78,2009-02-02T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.000000e+07,1,0,12/11/2009
6,22380,Estado,Minas Gerais,31,MG,00000.000000/2031-07,Arquivado pela STN,17944.000547/2016-23,2016-12-21T02:00:00Z,Operação contratual interna (com garantia da U...,Aditivo contratual,Instituição Financeira Nacional,Banco do Brasil S/A,Real,6.985602e+08,0,0,16/10/2017
7,5582,Estado,Minas Gerais,31,MG,None,Arquivado a pedido,17944.000148/2013-10,2013-04-02T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,3.000000e+08,0,0,10/04/2013
8,13595,Estado,Minas Gerais,31,MG,None,Indeferido,19405.000033/2002-03,2002-05-28T03:00:00Z,Operação contratual interna,Renegociação de dívidas,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,5.377000e+07,0,0,03/07/2002
9,7002,Estado,Minas Gerais,31,MG,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000477/2010-18,2010-05-24T03:00:00Z,Operação contratual interna (com garantia da U...,PEF - Programa Emergencial de Financiamento do...,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.672700e+08,1,0,17/06/2010


In [78]:
# 2) Seu código aqui
mg = busca_uf_interessado('MG', 'E')
mg['status'].value_counts()['Arquivado por decurso de prazo']

1

In [79]:
# 3) Seu código aqui
ba = busca_uf_interessado('BA', 'M')
print('Munincipio: ' + ba['interessado'].value_counts().index.to_list()[0] + 
      ' , Quantidade: ' + ba['interessado'].value_counts()[0].astype(str))

Munincipio: Camaçari , Quantidade: 19


In [81]:
# 4) Seu código aqui
ba_estado = busca_uf_interessado('BA', 'E')
ba_estado.to_csv('ba_estado.csv', index = False)